In [1]:
# Hidden using cell tags!
# Normally, you dont need to set this variable if you are using
# the default deployment of the backend
import os

os.environ["SNIP_DEPLOYMENT_URL"] = "https://snip:4000"
os.environ["SNIP_ADDITIONAL_REQUEST_ARGS"] = '{"verify": false}'

# Hide InsecureRequestWarning
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Book Endpoints

The python package provides a simple wrapper around the [books api](https://snip.roentgen.physik.uni-goettingen.de/apidocs/books). All API methods need either an account token or a book specific token. To find which method needs which token please check the call signature of all functions below.


## Retrieve all books for a user

Once you have created an `AccountToken`, you can retrieve all books the user has access to with the `get_books` function.

In [2]:
from snip.token import get_all_tokens, AccountToken

# Get account token
tokens = get_all_tokens()
account_tokens= list(filter(
    lambda t: isinstance(t, AccountToken), tokens[0]
))
if len(account_tokens) == 0:
    raise ValueError("No account token found. Please create one first.")
account_token: AccountToken = account_tokens[0]


We can retrieve all books for the user associated with the `get_books` function.

In [7]:
from snip.api.books import get_books

all_books = get_books(account_token)

## Retrieve a single book by its ID

You may also retrieve a single book by its ID using the `get_book` function. This can be done with both `AccountToken` and `BookToken`.


In [8]:
from snip.api.books import get_book

book = get_book(account_token, 1)
book

{'id': 1,
 'title': 'test',
 'comment': None,
 'created': datetime.datetime(2025, 6, 18, 10, 23, 12, tzinfo=datetime.timezone.utc),
 'finished': None,
 'last_updated': datetime.datetime(2025, 7, 31, 10, 31, 21, tzinfo=datetime.timezone.utc),
 'cover_page_id': None,
 'owner_id': 1,
 'owner_name': 'sebastian@mohrenclan.de',
 'owner_type': 'user',
 'default_background_type_id': 1,
 'background_type_name': 'blank',
 'background_type_description': 'Just your ordinary blank pages, nothing too special. Just a clean slate for you to write and draw onto.',
 'num_pages': 3}

## Create a new book

You may also create a new book using the `create_book` function. This requires an `AccountToken`.

In [9]:
from snip.api.books import create_book

new_book = create_book(account_token, "My New Book", "This is a comment.")
new_book

{'id': 5,
 'title': 'My New Book',
 'comment': 'This is a comment.',
 'created': datetime.datetime(2025, 9, 29, 11, 0, 27, tzinfo=datetime.timezone.utc),
 'finished': None,
 'last_updated': datetime.datetime(2025, 9, 29, 11, 0, 27, tzinfo=datetime.timezone.utc),
 'cover_page_id': None,
 'owner_id': 1,
 'owner_name': 'sebastian@mohrenclan.de',
 'owner_type': 'user',
 'default_background_type_id': 1,
 'background_type_name': 'blank',
 'background_type_description': 'Just your ordinary blank pages, nothing too special. Just a clean slate for you to write and draw onto.',
 'num_pages': 0}

## Collaborators

Book collaborators are all users and groups that have access to a specific book.

### Retrieve all collaborators for a book

You can retrieve all collaborators for a specific book using the `get_collaborators` function.

In [4]:
from snip.api.books import get_collaborators

collaborators = get_collaborators(account_token, 3)
collaborators

[{'type': 'user',
  'email': 'sebastian@mohrenclan.de',
  'pRead': True,
  'pWrite': True,
  'pDelete': True,
  'pACL': True,
  'id': 1,
  'self': False,
  'owner': True},
 {'type': 'group',
  'name': 'whatagroup',
  'pRead': True,
  'pWrite': True,
  'pDelete': False,
  'pACL': False,
  'id': 2,
  'owner': False}]

### Update collaborators for a book

We are able to update the collaborators for a specific book using the `update_collaborator` function. This function allows us to modify the permissions of existing collaborators or add new ones. You may also use this api to add groups as collaborators. For users please use the `invite_collaborator` function.

In [5]:
from snip.api.books import update_collaborator

collaborators[1]["pDelete"] = not collaborators[1]["pDelete"]

update_collaborator(
    account_token,
    3,
    collaborators[1]
)

### Remove a collaborator from a book

You can remove a collaborator from a book using the `remove_collaborator` function. 

In [6]:
from snip.api.books import remove_collaborator


remove_collaborator(
    account_token,
    3,
    collaborators[1]["id"],
    collaborators[1]["type"]
)

### Add a group

You can add a group as a collaborator to a book using the `update_collaborator` function. This function allows you to specify the group ID and the desired permissions for the group on the book.

In [7]:
from snip.api.books import update_collaborator

update_collaborator(
    account_token,
    3,
    {
        'id': 2,
        'type': 'group',
        'pRead': True,
        'pWrite': True,
        'pDelete': False,
        'pACL': False,
        'owner': False,
        'name': 'Test Group'
    }
)

### Invite a collaborator to a book

You can invite a new collaborator to a book using the `invite_collaborator` function. This function sends an invitation to the specified user to collaborate on the book with the given permissions.

In [3]:
from snip.api.books import  invite_collaborator

invite_collaborator(
    account_token,
    3,
    "sebastian.mohr@test.de"
)

{'success': True,
 'invites': [{'id': 7,
   'book_id': 3,
   'user_id': 2,
   'invited_by': 1,
   'created_at': '2025-09-29T11:16:08.000Z',
   'expire_at': '2025-10-01T11:16:08.000Z',
   'accepted_at': None,
   'emails': ['sebastian.mohr@test.de'],
   'emails_verified': [True],
   'credential_ids': [2],
   'credential_types': ['password'],
   'primary_credential_id': 2,
   'invited_by_emails': ['sebastian@mohrenclan.de'],
   'invited_by_emails_verified': [True],
   'invited_by_credential_ids': [1],
   'invited_by_credential_types': ['password'],
   'invited_by_primary_credential_id': 1}]}